In [1]:
import numpy as np
import pandas as pd
pd.__version__

'2.2.3'

In [2]:
import sklearn
sklearn.__version__

'1.6.1'

## Download the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.

In [3]:
# January 2023
df_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [4]:
df_jan.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0


## Q1: How many columns does the January 2023 data have?

In [5]:
print(len(df_jan.columns))

19


## Q2: Compute Duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [6]:
def get_total_minutes(row):
    seconds = row.total_seconds()
    minutes = seconds / 60
    return minutes
    
duration = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
duration = duration.apply(lambda x: get_total_minutes(x))

In [7]:
duration

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Length: 3066766, dtype: float64

In [8]:
print(f"The standard deviation is: {np.std(duration)}")

The standard deviation is: 42.59434429744777


## Q3: Dropping Outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [9]:
is_no_outlier = (duration >= 1) & (duration <= 60)
duration_no_outlier = duration[is_no_outlier]
fraction = np.round(len(duration_no_outlier)/len(duration),2) * 100
print(f"The fraction of the record left after dropping is: {fraction} %")

The fraction of the record left after dropping is: 98.0 %


In [10]:
df_jan["duration"] = duration
df_jan = df_jan.loc[is_no_outlier].reset_index(drop=True)

In [11]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009173 entries, 0 to 3009172
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Q4: One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

   - Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
   - Fit a dictionary vectorizer
   - Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [12]:
categorical = ["PULocationID","DOLocationID"]
df_jan[categorical] = df_jan.loc[:, categorical].astype(str)
df_jan[categorical].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3009173 entries, 0 to 3009172
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 45.9+ MB


In [13]:
features_dict = df_jan[categorical].to_dict(orient="records")

In [14]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(features_dict)


In [15]:
print(X_train.shape[1])

515


In [16]:
dv.feature_names_[:10]

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11']

## Q5: Training a model
Now let's use the feature matrix from the previous step to train a model.

    - Train a plain linear regression model with default parameters, where duration is the response variable
    - Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [18]:
y_train = df_jan["duration"].values

In [19]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [20]:
y_pred = model.predict(X_train)

In [21]:
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 7.6492624397080675


## Q6: Evaluate the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [22]:
def read_and_preprocess_data(filename):
    df = pd.read_parquet(filename)
    categorical = ["PULocationID","DOLocationID"]
    
    duration = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    duration = duration.apply(lambda x: get_total_minutes(x))
    df["duration"] = duration

    is_no_outlier = (duration >= 1) & (duration <= 60)
    df = df.loc[is_no_outlier].reset_index(drop=True)

    df[categorical] = df.loc[:, categorical].astype(str)
    
    return df   

In [23]:
filename = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet" # February 2023
df_val = read_and_preprocess_data(filename)

In [24]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2855951 entries, 0 to 2855950
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               float64       
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           float64       
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [25]:
val_features_dict = df_val[categorical].to_dict(orient="records")

In [33]:
X_val = dv.transform(val_features_dict)

In [27]:
X_val.shape[1]

515

In [28]:
y_pred = model.predict(X_val)

In [29]:
y_val = df_val["duration"].values

In [30]:
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

RMSE: 13.328465316947057


In [31]:
rmse_check = np.sqrt(np.mean((y_val - y_pred)**2))
print("RMSE:", rmse_check)

RMSE: 13.328465316947057
